In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.ensemble_model_backuo as em 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer

In [3]:

df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df
df['command'] = df['diff'].apply(lambda x : ' '.join(x.split('@@')[:2]))
df


/tmp/ipykernel_1203/2050868030.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,label,message,diff,command
0,0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...,diff --git a/src/kadmin/server/server_stubs.c ...
2,1,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...,diff --git a/spring-security-oauth2/src/main/j...
3,1,Release 2.72.4+171110,diff --git a/application/config/version.php b/...,diff --git a/application/config/version.php b/...
4,0,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...,...
10107,0,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....,diff --git a/les/benchmark.go b/les/benchmark....
10108,1,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,0,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,0,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...,diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [5]:
df['command'][0]

'diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm\nindex b6b498d..2450158 100644\n--- a/lib/DJabberd.pm\n+++ b/lib/DJabberd.pm\n  -173,6 +173,11 '

In [ ]:
# 设置数据缺失的情况
# 设置随机种子以便结果可重复
np.random.seed(42)

# 定义缺失值的比例
missing_rate = 0.2

# 计算每列需要创建缺失值的数量
num_missing = int(missing_rate * len(df))

# 随机选择行进行缺失
for col in ['message', 'code change']:
    missing_indices = np.random.choice(df.index, num_missing, replace=False)
    
    # 选择每个单元格的部分字符来替换，而不是全部替换成NaN
    for idx in missing_indices:
        if np.random.rand() > 0.5:
            # 保留前半部分
            df.loc[idx, col] = df.loc[idx, col][:len(df.loc[idx, col])//2] + '...'
        else:
            # 保留后半部分
            df.loc[idx, col] = '...' + df.loc[idx, col][len(df.loc[idx, col])//2:]

In [ ]:
df_dataset = []
for index, row in df.iterrows():
  df_dataset.append([row['message'],row['command'],row['label']])

In [3]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/bert-base-cased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/models/bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# Split the dataset
# preprocesser = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer)
train_data, val_data = train_test_split(df_dataset, test_size=0.2, random_state=1)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer)
val_dataset = preprocesser.SentencePairDataset(val_data, bert_tokenizer, codebert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)




# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)



base_model1 = em.BaseModel(bert_model,bert_tokenizer)
base_model2 = em.BaseModel(codebert_model,codebert_tokenizer)

# Create stacking model
stacking_model = em.StackingModel(base_model1, base_model2)
# Train the model
stacking_model.trainer(train_loader, val_loader,num_epochs=10, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


=============================train========================
Epoch 1/10, Loss: 0.368247739103781
=============================eval========================
Validation Accuracy: 0.8863074641621355
Precision: 0.8805213462991721
Recall: 0.8786386707735022
F1-Score: 0.8795549052985592
Epoch 1/10, Validation Accuracy: 0.8863074641621355
当前best acc 0.8863074641621355
=============================train========================
Epoch 2/10, Loss: 0.2021771462301985
=============================eval========================
Validation Accuracy: 0.9016312407315867
Precision: 0.9039858002764891
Recall: 0.8869607970731566
F1-Score: 0.8939636142193803
Epoch 2/10, Validation Accuracy: 0.9016312407315867
当前best acc 0.9016312407315867
=============================train========================
Epoch 3/10, Loss: 0.11112317853476558
=============================eval========================
Validation Accuracy: 0.8477508650519031
Precision: 0.8420412723449001
Recall: 0.860175986580481
F1-Score: 0.84444304636819